In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm
from torch import nn, optim
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from scipy.stats import norm

# Set locale environment variables
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'

# Install the required packages
!pip install torch torchvision timm


In [ ]:
# Paths to your datasets
train_dir = '/content/drive/MyDrive/Deep learning/train_data/Graph Patterns'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # CrossViT input size
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Load the dataset
dataset = datasets.ImageFolder(train_dir, transform=transform)

# Split into training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Print class names
class_names = dataset.classes
print("Classes:", class_names)


Classes: ['Ascending Triangle', 'Cup and Handle', 'Descending Triangle', 'Double Bottom', 'Double Top', 'Falling Wedge', 'Flag', 'Head and Shoulders Top', 'Inverse Head and Shoulder', 'Rounding', 'Symmetrical Triangle', 'Triple Bottom Reversal']


In [ ]:
class UnlabeledImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Custom dataset for loading images without labels.

        Args:
            root_dir (str): Path to the directory containing images.
            transform (callable, optional): A function/transform to apply to the images.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(root_dir, fname)
            for fname in os.listdir(root_dir)
            if fname.endswith((".jpg", ".jpeg", ".png"))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path  # Return image and its path for reference

# Define test directory
test_dir = '/content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos'

# Create test dataset and DataLoader
test_dataset = UnlabeledImageDataset(root_dir=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

print(f"Loaded {len(test_dataset)} test images.")


Loaded 62 test images.


In [ ]:
# Number of classes
num_classes = len(class_names)

# Load CrossViT model
model = timm.create_model('crossvit_15_240', pretrained=True, num_classes=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on:", device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded on: cuda


In [ ]:
# Define optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()

def train_model(model, train_loader, val_loader, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss /= len(train_loader.dataset)
        train_acc = correct / total

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader.dataset)
        val_acc = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Train the model
train_model(model, train_loader, val_loader, num_epochs=100)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/100, Train Loss: 2.5354, Train Acc: 0.0533, Val Loss: 2.5103, Val Acc: 0.0263


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 2/100, Train Loss: 2.5118, Train Acc: 0.0667, Val Loss: 2.5046, Val Acc: 0.0263


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch 98/100, Train Loss: 0.0364, Train Acc: 0.9933, Val Loss: 6.2398, Val Acc: 0.0789


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 99/100, Train Loss: 0.0272, Train Acc: 0.9933, Val Loss: 6.2579, Val Acc: 0.0789


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 100/100, Train Loss: 0.0301, Train Acc: 0.9933, Val Loss: 6.2501, Val Acc: 0.1316


In [ ]:
def test_model_on_unlabeled_data(model, test_loader, class_names):
    """
    Perform inference on unlabeled test images.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for test dataset.
        class_names: List of class names corresponding to class indices.

    Returns:
        predictions: List of tuples (image_path, predicted_class, confidence_score).
    """
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs, img_paths in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            conf, predicted = torch.max(probabilities, 1)  # Get predicted class and confidence
            for i in range(len(img_paths)):
                predictions.append((img_paths[i], class_names[predicted[i].item()], conf[i].item()))

    return predictions

# Run inference
predictions = test_model_on_unlabeled_data(model, test_loader, class_names)

# Print results
for img_path, pred_class, conf in predictions:
    print(f"Image: {img_path} | Predicted Class: {pred_class} | Confidence: {conf:.2f}")


Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_3061_to_3150.png | Predicted Class: Cup and Handle | Confidence: 0.95
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_2161_to_2250.png | Predicted Class: Descending Triangle | Confidence: 0.65
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_181_to_270.png | Predicted Class: Cup and Handle | Confidence: 0.64
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_2431_to_2520.png | Predicted Class: Ascending Triangle | Confidence: 0.92
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_5401_to_5490.png | Predicted Class: Symmetrical Triangle | Confidence: 0.97
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_4051_to_4140.png | Predicted Class: Descending Triangle | Confidence: 0.49
Image: /content/drive/MyDrive/Deep learning/test_data/Nvid

In [ ]:
def compute_confidence_scores_and_intervals(model, data_loader, class_names, confidence_interval=0.95):
    """
    Computes confidence scores for predictions and calculates confidence intervals.

    Args:
        model (torch.nn.Module): Trained model for inference.
        data_loader (DataLoader): DataLoader for the test dataset.
        class_names (list): List of class names corresponding to class indices.
        confidence_interval (float): Desired confidence interval (default is 0.95).

    Returns:
        dict: Contains:
            - predictions: List of tuples (image_path, predicted_class, confidence_score).
            - confidence_summary: Dictionary with mean, min, max, and confidence interval bounds.
    """
    model.eval()
    predictions = []
    confidence_scores = []

    with torch.no_grad():
        for inputs, img_paths in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Compute probabilities
            conf, predicted = torch.max(probabilities, 1)  # Confidence and predicted class

            for i in range(len(img_paths)):
                score = conf[i].item()
                confidence_scores.append(score)
                predictions.append((img_paths[i], class_names[predicted[i].item()], score))

    # Compute statistics for confidence scores
    mean_conf = np.mean(confidence_scores)
    std_dev = np.std(confidence_scores, ddof=1)
    n = len(confidence_scores)
    z = norm.ppf((1 + confidence_interval) / 2)  # Z-score for the confidence interval
    margin_of_error = z * (std_dev / np.sqrt(n))
    lower_bound = mean_conf - margin_of_error
    upper_bound = mean_conf + margin_of_error

    confidence_summary = {
        "mean_confidence": mean_conf,
        "min_confidence": np.min(confidence_scores),
        "max_confidence": np.max(confidence_scores),
        "confidence_interval": (lower_bound, upper_bound),
    }

    return {"predictions": predictions, "confidence_summary": confidence_summary}


In [ ]:
results = compute_confidence_scores_and_intervals(model, test_loader, class_names)

# Extract predictions and confidence summary
predictions = results["predictions"]
confidence_summary = results["confidence_summary"]

# Print predictions
for img_path, pred_class, conf in predictions:
    print(f"Image: {img_path} | Predicted Class: {pred_class} | Confidence: {conf:.2f}")

# Print confidence summary
print("\nConfidence Summary:")
print(f"Mean Confidence: {confidence_summary['mean_confidence']:.2f}")
print(f"Min Confidence: {confidence_summary['min_confidence']:.2f}")
print(f"Max Confidence: {confidence_summary['max_confidence']:.2f}")
print(f"Confidence Interval (95%): {confidence_summary['confidence_interval']}")


Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_3061_to_3150.png | Predicted Class: Cup and Handle | Confidence: 0.95
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_2161_to_2250.png | Predicted Class: Descending Triangle | Confidence: 0.65
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_181_to_270.png | Predicted Class: Cup and Handle | Confidence: 0.64
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_2431_to_2520.png | Predicted Class: Ascending Triangle | Confidence: 0.92
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_5401_to_5490.png | Predicted Class: Symmetrical Triangle | Confidence: 0.97
Image: /content/drive/MyDrive/Deep learning/test_data/Nvidia_3Mos/NVDA_2000_to_2021_4051_to_4140.png | Predicted Class: Descending Triangle | Confidence: 0.49
Image: /content/drive/MyDrive/Deep learning/test_data/Nvid